# Analyze city property inventory

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

### Statistics by `parceltype`

In [3]:
propertiespt = pd.read_sql_query("""SELECT parceltype, COUNT(DISTINCT parceleproppgh.parcelid), SUM(lotarea::int) AS lotarea
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
--WHERE parcelstatus = 'Available for Sale'
GROUP BY parceltype
ORDER BY count DESC;""", DB)

# Merge permanent city ownership parcel types
propertiespt.loc[propertiespt.parceltype.isin([
    'Greenway', 'Park', 'Infrastructure Protection', 'City Facility', 'Other Public Use'
]), 'parceltype'] = 'PCO'
propertiespt = propertiespt.groupby(propertiespt.parceltype).aggregate({ 'count': 'sum', 'lotarea': 'sum' }).sort_values(by='count')

# Convert lot area to acres
propertiespt['lotarea'] = propertiespt.lotarea / 43560

# Percent of city neighborhood acreage per parcel type
propertiespt['pct_city'] = 100 * (propertiespt.lotarea / 33161.843)

# Multiple of Schenley Park (somewhat recognizable comparison)
propertiespt['mult_schenley'] = propertiespt.lotarea / 456

propertiespt

,count,lotarea,pct_city,mult_schenley
parceltype,,,,
PLB Transfer,3,0.225367,0.000680,0.000494
Unknown Public Use,10,0.529982,0.001598,0.001162
Unknown,180,83.621671,0.252162,0.183381
CDC Property Reserve,192,11.389555,0.034345,0.024977
PCO,2240,2287.426102,6.897765,5.016285
Public Sale,2712,225.179522,0.679032,0.493815
URA Transfer,2815,200.797153,0.605507,0.440345
Hold For Study,4991,2748.636042,8.288550,6.027711
